# Importing packages

In [3]:
import requests
import pandas as pd
import mysql.connector
import json

In this ETL process we will be working with fetching COVID data from and API, transforming and loading into a MSQL database
    * API is a free covid API of results from the USA
    * MYSQL server to be used is a localhost server on my laptop

In [51]:
#fetching data from API
URL="https://api.covidtracking.com/v1/states/ca/current.json"
covid_data=requests.get(URL)
covid_data

<Response [200]>

In [52]:
data = json.loads(covid_data.text)
data

{'date': 20210307,
 'state': 'CA',
 'positive': 3501394,
 'probableCases': None,
 'negative': None,
 'pending': None,
 'totalTestResultsSource': 'totalTestsViral',
 'totalTestResults': 49646014,
 'hospitalizedCurrently': 4291,
 'hospitalizedCumulative': None,
 'inIcuCurrently': 1159,
 'inIcuCumulative': None,
 'onVentilatorCurrently': None,
 'onVentilatorCumulative': None,
 'recovered': None,
 'lastUpdateEt': '3/7/2021 02:59',
 'dateModified': '2021-03-07T02:59:00Z',
 'checkTimeEt': '03/06 21:59',
 'death': 54124,
 'hospitalized': None,
 'hospitalizedDischarged': None,
 'dateChecked': '2021-03-07T02:59:00Z',
 'totalTestsViral': 49646014,
 'positiveTestsViral': None,
 'negativeTestsViral': None,
 'positiveCasesViral': 3501394,
 'deathConfirmed': None,
 'deathProbable': None,
 'totalTestEncountersViral': None,
 'totalTestsPeopleViral': None,
 'totalTestsAntibody': None,
 'positiveTestsAntibody': None,
 'negativeTestsAntibody': None,
 'totalTestsPeopleAntibody': None,
 'positiveTestsPeopl

# Transform the Data from Json to a Pandas Dataframe

In [53]:
data=pd.DataFrame(data, index=[0])
data

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,CA,3501394,None,None,None,totalTestsViral,49646014,4291,None,...,None,258,0,63c5c0fd2daef2fb65150e9db486de98ed3f7b72,0,0,0,0,0,


# Load 
We will load the data into a MySQL database on a localhost server

In [54]:
# Connecting the MySQL server
mysqldb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="goodies4SQL@",
  database="covidtracking"
)

print(mysqldb)

In [63]:
mycursor = mysqldb.cursor()
mycursor.execute("DROP TABLE if exists covidtrack")

In [64]:
#Creating the table to Load the Pandas Dataframe
mycursor.execute("CREATE TABLE covidtrack(date int, state varchar(2), positive int, probableCases int, negative int, pending int, totalTestResultsSource varchar(25), totalTestResults int, hospitalizedCurrently int, hospitalizedCumulative int, inIcuCurrently int, inIcuCumulative int, onVentilatorCurrently int, onVentilatorCumulative int, recovered int, lastUpdateEt datetime, dateModified datetime, checkTimeEt datetime, death int, hospitalized varchar(25), hospitalizedDischarged varchar(25), dateChecked datetime, totalTestsViral int, positiveTestsViral int, negativeTestsViral int, positiveCasesViral int, deathConfirmed Varchar(8), deathProbable Varchar(8), totalTestEncountersViral Varchar(8), totalTestsPeopleViral Varchar(8), totalTestsAntibody Varchar(8), positiveTestsAntibody Varchar(8), negativeTestsAntibody Varchar(8), totalTestsPeopleAntibody Varchar(8), positiveTestsPeopleAntibody Varchar(8), negativeTestsPeopleAntibody Varchar(8), totalTestsPeopleAntigen Varchar(8), positiveTestsPeopleAntigen Varchar(8), totalTestsAntigen Varchar(8), positiveTestsAntigen Varchar(8), fips int, positiveIncrease int, negativeIncrease int, total int, totalTestResultsIncrease int, posNeg int, dataQualityGrade Varchar(8), deathIncrease int, hospitalizedIncrease int, hash varchar(255), commercialScore int, negativeRegularScore int, negativeScore int, positiveScore int, score int, grade int )")

In [ ]:
data.to_sql('covidtrack', mysqldb, index=False)